In [1]:
import keras.backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

import cnn_builder as cbuild
import cnn_methods as cfunc
import config
import csv
import helper_fxns as hf
import importlib
import matplotlib.pyplot as plt
import numpy as np
import operator
import os
import pandas as pd
import random
%matplotlib inline

Using TensorFlow backend.


In [2]:
importlib.reload(config)
importlib.reload(cfunc)
importlib.reload(hf)
importlib.reload(cbuild)
C = config.Config()

In [18]:
model = cbuild.build_2d_cnn(C, 'adam')#Adam(lr=0.01))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 36, 36, 1)     0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 36, 36, 1)     0                                            
____________________________________________________________________________________________________
input_7 (InputLayer)             (None, 36, 36, 1)     0                                            
____________________________________________________________________________________________________
conv2d_5 (Conv2D)                (None, 34, 34, 64)    640         input_5[0][0]                    
___________________________________________________________________________________________

model = cbuild.run_cnn(model, C)

In [19]:
nb_classes = len(C.classes_to_include)
voi_df = pd.read_csv(C.art_voi_path)
intensity_df = pd.read_csv(C.int_df_path)
#intensity_df.loc[intensity_df["art_int"] == 0, "art_int"] = np.mean(intensity_df[intensity_df["art_int"] > 0]["art_int"])
#intensity_df.loc[intensity_df["ven_int"] == 0, "ven_int"] = np.mean(intensity_df[intensity_df["ven_int"] > 0]["ven_int"])
#intensity_df.loc[intensity_df["eq_int"] == 0, "eq_int"] = np.mean(intensity_df[intensity_df["eq_int"] > 0]["eq_int"])

orig_data_dict, num_samples = cfunc.collect_unaug_data(C.classes_to_include, C, voi_df, intensity_df)
print(num_samples)

avg_X2 = {}
for cls in C.classes_to_include:
    avg_X2[cls] = np.mean(orig_data_dict[cls][1], axis=0)

{'cyst': 74, 'colorectal': 124, 'cholangio': 56, 'hemangioma': 56, 'hcc': 89, 'fnh': 67}


In [20]:
train_ids = {} #filenames of training set originals
test_ids = {} #filenames of test set
X_test = []
X2_test = []
Y_test = []
Z_test = []
X_train_orig = []
X2_train_orig = []
Y_train_orig = []
Z_train_orig = []

train_samples = {}

for cls_num, cls in enumerate(orig_data_dict):
    cls_num = C.classes_to_include.index(cls)

    train_samples[cls] = round(num_samples[cls]*C.train_frac)

    order = np.random.permutation(list(range(num_samples[cls])))
    train_ids[cls] = list(orig_data_dict[cls][2][order[:train_samples[cls]]])
    test_ids[cls] = list(orig_data_dict[cls][2][order[train_samples[cls]:]])

    X_test = X_test + list(orig_data_dict[cls][0][order[train_samples[cls]:]])
    X2_test = X2_test + list(orig_data_dict[cls][1][order[train_samples[cls]:]])
    Y_test = Y_test + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (num_samples[cls] - train_samples[cls])
    Z_test = Z_test + test_ids[cls]

    X_train_orig = X_train_orig + list(orig_data_dict[cls][0][order[:train_samples[cls]]])
    X2_train_orig = X2_train_orig + list(orig_data_dict[cls][1][order[:train_samples[cls]]])
    Y_train_orig = Y_train_orig + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (train_samples[cls])
    Z_train_orig = Z_train_orig + train_ids[cls]

    print("%s has %d samples for training (%d after augmentation) and %d for testing" %
          (cls, train_samples[cls], train_samples[cls] * C.aug_factor, num_samples[cls] - train_samples[cls]))

#Y_test = np_utils.to_categorical(Y_test, nb_classes)
#Y_train_orig = np_utils.to_categorical(Y_train_orig, nb_classes)
X_test = [np.array(X_test), np.array(X2_test)]
X_train_orig = [np.array(X_train_orig), np.array(X2_train_orig)]

Y_test = np.array(Y_test)
Y_train_orig = np.array(Y_train_orig)

Z_test = np.array(Z_test)
Z_train_orig = np.array(Z_train_orig)

cyst has 59 samples for training (2950 after augmentation) and 15 for testing
colorectal has 99 samples for training (4950 after augmentation) and 25 for testing
cholangio has 45 samples for training (2250 after augmentation) and 11 for testing
hemangioma has 45 samples for training (2250 after augmentation) and 11 for testing
hcc has 71 samples for training (3550 after augmentation) and 18 for testing
fnh has 54 samples for training (2700 after augmentation) and 13 for testing


train_generator = train_generator_func()
model_pretrain.fit_generator(train_generator, steps_per_epoch=120, epochs=50)#, callbacks=[early_stopping])

In [ ]:
#early_stopping = EarlyStopping(monitor='acc', min_delta=0.01, patience=4)
train_generator = cbuild.train_generator_func_2d(C, train_ids, intensity_df, voi_df, avg_X2, n=6, n_art=4)
model.fit_generator(train_generator, steps_per_epoch=30, epochs=75)#, callbacks=[early_stopping])

Epoch 1/75
30/30 [==============================] - 14s - loss: 1.5668 - acc: 0.3128    
Epoch 2/75
30/30 [==============================] - 12s - loss: 1.1859 - acc: 0.4978    
Epoch 3/75
30/30 [==============================] - 12s - loss: 1.0175 - acc: 0.5711    
Epoch 4/75
30/30 [==============================] - 11s - loss: 0.9033 - acc: 0.6344    
Epoch 5/75
30/30 [==============================] - 12s - loss: 0.8459 - acc: 0.6639    
Epoch 6/75
30/30 [==============================] - 10s - loss: 0.8123 - acc: 0.6728    
Epoch 7/75
30/30 [==============================] - 10s - loss: 0.7501 - acc: 0.6961    
Epoch 8/75
30/30 [==============================] - 10s - loss: 0.7260 - acc: 0.7261    
Epoch 9/75
30/30 [==============================] - 10s - loss: 0.6904 - acc: 0.7211    
Epoch 10/75
30/30 [==============================] - 10s - loss: 0.6332 - acc: 0.7422    
Epoch 11/75
30/30 [==============================] - 9s - loss: 0.6656 - acc: 0.7361     
Epoch 12/75
30/30 [

hist = model.fit(X_train2, Y_train2, batch_size=32, epochs=200, validation_data=(X_test, Y_test), callbacks=[early_stopping])

In [15]:
voi_df_art = pd.read_csv(C.art_voi_path)
voi_df_ven = pd.read_csv(C.ven_voi_path)
voi_df_eq = pd.read_csv(C.eq_voi_path)

with open(C.small_voi_path, 'r') as csv_file:
    reader = csv.reader(csv_file)
    small_vois = dict(reader)
    
for key in small_vois:
    small_vois[key] = [int(x) for x in small_vois[key][1:-1].split(', ')]

In [9]:
C.classes_to_include

['hcc', 'cholangio', 'colorectal', 'cyst', 'hemangioma', 'fnh']

In [49]:
importlib.reload(cfunc)

<module 'cnn_methods' from 'C:\\Users\\Clinton\\Documents\\voi-classifier\\cnn_methods.py'>

In [8]:
X_test = cfunc.separate_phases_2d(X_test)
X_train_orig = cfunc.separate_phases_2d(X_train_orig)

In [9]:
C.output_img_dir = "Z:\\OUTPUT\\12-07-2d"

In [16]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

cfunc.save_output(Z_test, y_pred, y_true, voi_df_art, small_vois, C.classes_to_include, C)

In [17]:
Y_pred = model.predict(X_train_orig)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_train_orig])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

cfunc.save_output(Z_train_orig, y_pred, y_true, voi_df_art, small_vois, C.classes_to_include, C)

In [54]:
cm

array([[ 7,  0,  0,  0,  0,  3],
       [ 0, 10,  1,  0,  0,  0],
       [ 0,  0, 10,  0,  0,  0],
       [ 0,  0,  0, 15,  0,  0],
       [ 0,  0,  0,  1, 10,  0],
       [ 0,  0,  0,  0,  0, 13]], dtype=int64)

In [12]:
y_true_simp, y_pred_simp, _ = cfunc.condense_cm(y_true, y_pred, C.classes_to_include)

In [13]:
accuracy_score(y_true_simp, y_pred_simp)

0.83870967741935487

In [11]:
accuracy_score(y_true, y_pred)

0.77419354838709675

4 cls: 94/94%, 86/88%, 86%/90%
5 cls: 80%/89%, 72%/86%, 84%/86%

6 cls no artificial, separate phases: 75%/84%
6 cls with artificial, separate phases: 77%/84%

fn_list = Z_test[~np.equal(y_pred, y_true)]
wrong_guesses = np.array(y_pred)[~np.equal(y_pred, y_true)]

activ = model2.predict(X_train)
#activ = model2.predict(np.expand_dims(X_train[10],axis=0))

import operator
#print("Ground truth:", [C.classes_to_include[max(enumerate(x), key=operator.itemgetter(1))[0]] for x in Y_val[::30]])
Y_ = model.predict(X_test)
print("Predictions:", [C.classes_to_include[max(enumerate(x), key=operator.itemgetter(1))[0]] + " (%.5f%% probability)" % (max(x)*100) for x in Y_])